In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow.keras.utils import to_categorical
import os
from transformers import *
print(tf.__version__)

2.0.0-rc1


In [2]:
TRAIN_PATH = './data/train_dataset/'
TEST_PATH = './data/test_dataset/'
BERT_PATH = './bert_base_chinese/'
MAX_SEQUENCE_LENGTH = 140
input_categories = '微博中文内容'
output_categories = '情感倾向'

df_train = pd.read_csv(TRAIN_PATH+'nCoV_100k_train.labled.csv',engine ='python')
df_train = df_train[df_train[output_categories].isin(['-1','0','1'])]
df_test = pd.read_csv(TEST_PATH+'nCov_10k_test.csv',engine ='python')
df_sub = pd.read_csv(TEST_PATH+'submit_example.csv')
print('train shape =', df_train.shape)
print('test shape =', df_test.shape)

train shape = (99913, 7)
test shape = (10000, 6)


In [3]:
def _convert_to_transformer_inputs(instance, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for transformer (including bert)"""
    
    def return_id(str1, truncation_strategy, length):

        inputs = tokenizer.encode_plus(str1,
            add_special_tokens=True,
            max_length=length,
            truncation_strategy=truncation_strategy)
        
        input_ids =  inputs["input_ids"]
        input_masks = [1] * len(input_ids)
        input_segments = inputs["token_type_ids"]
        padding_length = length - len(input_ids)
        padding_id = tokenizer.pad_token_id
        input_ids = input_ids + ([padding_id] * padding_length)
        input_masks = input_masks + ([0] * padding_length)
        input_segments = input_segments + ([0] * padding_length)
        
        return [input_ids, input_masks, input_segments]
    
    input_ids, input_masks, input_segments = return_id(instance, 'longest_first', max_sequence_length)
    
    return [input_ids, input_masks, input_segments]

def compute_input_arrays(df, columns, tokenizer, max_sequence_length):
    input_ids, input_masks, input_segments = [], [], []
    for instance in tqdm(df[columns]):
        
        ids, masks, segments = _convert_to_transformer_inputs(str(instance), tokenizer, max_sequence_length)
        
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)

    return [np.asarray(input_ids, dtype=np.int32), 
            np.asarray(input_masks, dtype=np.int32), 
            np.asarray(input_segments, dtype=np.int32)
           ]

In [4]:
tokenizer = BertTokenizer.from_pretrained(BERT_PATH+'bert-base-chinese-vocab.txt')
inputs = compute_input_arrays(df_train, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)
test_inputs = compute_input_arrays(df_test, input_categories, tokenizer, MAX_SEQUENCE_LENGTH)

Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated
100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:12<00:00, 821.93it/s]


In [5]:
def compute_output_arrays(df, columns):
    return np.asarray(df[columns].astype(int) + 1)
outputs = compute_output_arrays(df_train, output_categories)

# BERT模型
<img src="https://imgkr.cn-bj.ufileos.com/9115ac01-f455-498b-8c38-9c4abb04046c.png" width="50%" height="50%" />

# 修改模型
- bert 修改
- 加入 LSTM、GRU等作为Encoder

In [9]:
class WeiboBERT(tf.keras.Model):
    def __init__(self):
        super(WeiboBERT, self).__init__(name='Weibo_bert')
        config = BertConfig.from_pretrained(BERT_PATH + 'bert-base-chinese-config.json',output_hidden_states=True) 
        self.bert_model = TFBertModel.from_pretrained(BERT_PATH+'bert-base-chinese-tf_model.h5', config=config)
        self.concat =  tf.keras.layers.Concatenate(axis=2)
        self.avgpool = tf.keras.layers.GlobalAveragePooling1D()
        self.dropout = tf.keras.layers.Dropout(0.15)
        self.output_ = tf.keras.layers.Dense(3, activation='softmax')
   
        
    def call(self, inputs):
        input_id,input_mask,input_atn = inputs
        sequence_output, pooler_output, hidden_states  = self.bert_model(input_id, attention_mask=input_mask, token_type_ids=input_atn)
        h12 = tf.reshape(hidden_states[-1][:,0],(-1,1,768))
        h11 = tf.reshape(hidden_states[-2][:,0],(-1,1,768))
        h10 = tf.reshape(hidden_states[-3][:,0],(-1,1,768))
        h09 = tf.reshape(hidden_states[-4][:,0],(-1,1,768)) 
        concat_hidden = self.concat(([h12, h11, h10, h09]))
        x = self.avgpool(concat_hidden)
        x = self.dropout(x)
        x = self.output_(x)
        return x

In [ ]:
gkf = StratifiedKFold(n_splits=5).split(X=df_train[input_categories].fillna('-1'), y=df_train[output_categories].fillna('-1'))

valid_preds = []
test_preds = []
for fold, (train_idx, valid_idx) in enumerate(gkf):
    train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
    train_outputs = to_categorical(outputs[train_idx])

    valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
    valid_outputs = to_categorical(outputs[valid_idx])

    model = WeiboBERT()
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['acc']) 
    
    model.fit(train_inputs, train_outputs, validation_data= [valid_inputs, valid_outputs], epochs=2, batch_size=32)
    test_preds.append(model.predict(test_inputs))
    K.clear_session()

# 修改损失函数

In [16]:
def Focal_Loss(y_true, y_pred, alpha=0.5, gamma=2):
    """
    focal loss for multi-class classification
    fl(pt) = -alpha*(1-pt)^(gamma)*log(pt)
    :param y_true: ground truth one-hot vector shape of [batch_size, nb_class]
    :param y_pred: prediction after softmax shape of [batch_size, nb_class]
    :param alpha:
    :param gamma:
    :return:
    """
    y_pred += tf.keras.backend.epsilon()
    ce = -y_true * tf.math.log(y_pred)
    weight = tf.pow(1 - y_pred, gamma) * y_true
    fl = ce * weight * alpha
    reduce_fl = tf.keras.backend.max(fl, axis=-1)
    return reduce_fl

In [ ]:
gkf = StratifiedKFold(n_splits=5).split(X=df_train[input_categories].fillna('-1'), y=df_train[output_categories].fillna('-1'))

valid_preds = []
test_preds = []
for fold, (train_idx, valid_idx) in enumerate(gkf):
    train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
    train_outputs = to_categorical(outputs[train_idx])

    valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
    valid_outputs = to_categorical(outputs[valid_idx])

    model = WeiboBERT()
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    train_dataset = tf.data.Dataset.from_tensor_slices(((train_inputs[0],train_inputs[1],train_inputs[2]),train_outputs)).shuffle(buffer_size=1000).batch(32)
    valid_dataset = tf.data.Dataset.from_tensor_slices(((valid_inputs[0],valid_inputs[1],valid_inputs[2]),valid_outputs)).batch(32)
    
    FL=lambda y_true,y_pred: Focal_Loss(y_true, y_pred, alpha=0.25, gamma=2)
    
    model.compile(loss=FL, optimizer=optimizer, metrics=['acc'])   
    model.fit(train_inputs, train_outputs, validation_data= [valid_inputs, valid_outputs], epochs=2, batch_size=32)
    test_preds.append(model.predict(test_inputs))
    K.clear_session()

# 对抗训练

https://kexue.fm/archives/7234 

ADVERSARIAL TRAINING METHODS FOR SEMI-SUPERVISED TEXT CLASSIFICATION (https://arxiv.org/pdf/1605.07725.pdf)

In [25]:
def search_layer(inputs, name, exclude_from=None):
    """根据inputs和name来搜索层
    说明：inputs为某个层或某个层的输出；name为目标层的名字。
    实现：根据inputs一直往上递归搜索，直到发现名字为name的层为止；
         如果找不到，那就返回None。
    """
    if exclude_from is None:
        exclude_from = set()

    if isinstance(inputs, tf.keras.layers.Layer):
        layer = inputs
    else:
        layer = inputs._keras_history[0]

    if layer.name == name:
        return layer
    elif layer in exclude_from:
        return None
    else:
        exclude_from.add(layer)
        if isinstance(layer, tf.keras.models.Model):
            model = layer
            for layer in model.layers:
                if layer.name == name:
                    return layer
        inbound_layers = layer._inbound_nodes[0].inbound_layers
        if not isinstance(inbound_layers, list):
            inbound_layers = [inbound_layers]
        if len(inbound_layers) > 0:
            for layer in inbound_layers:
                layer = search_layer(layer, name, exclude_from)
                if layer is not None:
                    return layer

def sparse_categorical_crossentropy(y_true, y_pred):
    """自定义稀疏交叉熵
    这主要是因为keras自带的sparse_categorical_crossentropy不支持求二阶梯度。
    """
    y_true = K.reshape(y_true, K.shape(y_pred)[:-1])
    y_true = K.cast(y_true, 'int32')
    y_true = K.one_hot(y_true, K.shape(y_pred)[-1])
    return K.categorical_crossentropy(y_true, y_pred)


def loss_with_gradient_penalty(y_true, y_pred, epsilon=1):
    """带梯度惩罚的loss
    """
    loss = K.mean(sparse_categorical_crossentropy(y_true, y_pred))
    embeddings = search_layer(y_pred, 'Embedding-Token').embeddings
    gp = K.sum(K.gradients(loss, [embeddings])[0].values**2)
    return loss + 0.5 * epsilon * gp

In [ ]:
gkf = StratifiedKFold(n_splits=5).split(X=df_train[input_categories].fillna('-1'), y=df_train[output_categories].fillna('-1'))

valid_preds = []
test_preds = []
for fold, (train_idx, valid_idx) in enumerate(gkf):
    train_inputs = [inputs[i][train_idx] for i in range(len(inputs))]
    train_outputs = to_categorical(outputs[train_idx])

    valid_inputs = [inputs[i][valid_idx] for i in range(len(inputs))]
    valid_outputs = to_categorical(outputs[valid_idx])

    model = WeiboBERT()
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
    train_dataset = tf.data.Dataset.from_tensor_slices(((train_inputs[0],train_inputs[1],train_inputs[2]),train_outputs)).shuffle(buffer_size=1000).batch(32)
    valid_dataset = tf.data.Dataset.from_tensor_slices(((valid_inputs[0],valid_inputs[1],valid_inputs[2]),valid_outputs)).batch(32)
    
    
    model.compile(loss=loss_with_gradient_penalty, optimizer=optimizer, metrics=['acc'])   
    model.fit(train_dataset, validation_data= valid_dataset, epochs=2)
    test_preds.append(model.predict(test_inputs))
    K.clear_session()